<a href="https://colab.research.google.com/github/LEECHANGDAEE/finance/blob/main/xgb_all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader

In [2]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import xgboost as xgb

In [3]:
stock_list = pd.read_csv('Stock_List.csv',encoding='cp949')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [4]:
start_date = '20210104'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())


WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (220, 1)


,Date
0,2021-01-04
1,2021-01-05
2,2021-01-06
3,2021-01-07
4,2021-01-08


In [5]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample['weekday'] = sample.Date.apply(lambda x : x.weekday())
sample['weeknum'] = sample.Date.apply(lambda x : x.strftime('%V'))
sample.Close = sample.Close.ffill()
sample = pd.pivot_table(data = sample, values = 'Close', columns = 'weekday', index = 'weeknum')
sample.head()

weekday,0,1,2,3,4
weeknum,,,,,
01,83000.0,83900.0,82200.0,82900.0,88800.0
02,91000.0,90600.0,89700.0,89700.0,88000.0
03,85000.0,87000.0,87200.0,88100.0,86800.0
04,89400.0,86700.0,85600.0,83700.0,82000.0
05,83000.0,84400.0,84600.0,82500.0,83500.0


In [21]:
model = xgb.XGBRegressor(objective='reg:squarederror',
                             learning_rate =0.1,
                             n_estimators=1000,
                             max_depth=4,
                             gamma=0.1,  
)

In [7]:
model = LinearRegression()

In [22]:
x = sample.iloc[0:-2].to_numpy()
x.shape

(42, 5)

In [23]:
y = sample.iloc[1:-1].to_numpy()
y_0 = y[:,0]
y_1 = y[:,1]
y_2 = y[:,2]
y_3 = y[:,3]
y_4 = y[:,4]

y_values = [y_0, y_1, y_2, y_3, y_4]

In [24]:
x_public = sample.iloc[-2].to_numpy()

In [25]:
predictions = []
for y_value in y_values :
    model.fit(x,y_value)
    prediction = model.predict(np.expand_dims(x_public,0))
    predictions.append(prediction[0])
predictions

/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +


[70200.18, 71099.76, 70100.02, 70699.77, 69800.23]

In [26]:
sample.iloc[-1].values

array([69900., 71500., 70400., 70600., 70200.])

In [27]:
sample_submission = pd.read_csv('sample_submission.csv', encoding='cp949')

In [28]:
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    
    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 10월 22일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 10월 29일까지의 데이터를 학습한다.
    y_0 = y[:,0]
    y_1 = y[:,1]
    y_2 = y[:,2]
    y_3 = y[:,3]
    y_4 = y[:,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 11월 1일부터 11월 5일까지의 데이터를 예측할 것이다.
    
    predictions = []
    for y_value in y_values :
        model.fit(x,y_value)
        prediction = model.predict(np.expand_dims(x_public,0))
        predictions.append(prediction[0])
    sample_submission.loc[:,code] = predictions * 2
sample_submission.isna().sum().sum()

  0%|          | 0/370 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +
  0%|          | 1/370 [00:02<12:47,  2.08s/it]/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +
  1%|          | 2/370 [00:03<11:08,  1.82s/it]/usr/local/lib/python3.7/dist-packages/xgboost/core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +
  1%|          | 3/370 [00:

0

In [29]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [30]:
sample_submission.to_csv('xgb_reg.csv',index=False)

In [31]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29149.949219,34826.691406,60575.863281,146999.937500,107999.679688,16884.296875,48547.359375,85484.578125,102145.664062,682977.0000,50433.257812,247999.671875,35018.828125,57103.042969,17345.560547,2253.489990,27806.943359,5071.983887,331632.18750,50341.953125,13400.054688,33539.250000,9638.376953,8013.771973,30842.335938,143235.109375,47997.125000,87649.468750,44343.203125,247545.84375,282268.06250,83206.437500,105634.976562,32937.640625,30510.511719,34023.398438,153630.187500,210412.703125,97513.460938,...,39494.773438,107285.273438,290699.68750,12958.489258,163459.671875,176812.578125,7200.556152,7145.586426,34668.503906,13349.957031,77157.171875,43700.105469,52671.980469,26512.861328,296426.78125,618971.7500,716999.8750,23169.218750,63503.175781,62761.226562,24099.759766,118999.765625,36879.800781,12999.738281,3246.736328,39108.976562,18475.681641,65076.359375,97000.257812,5662.889160,50448.296875,50764.078125,86899.554688,36608.003906,26565.265625,55203.621094,325301.1875,262700.000000,27096.222656,17107.183594
1,2021-11-02,29552.576172,35203.156250,60889.988281,144500.281250,105499.710938,17170.509766,49360.031250,85147.671875,104140.140625,668904.6250,51579.695312,247499.750000,35585.570312,56455.562500,17541.794922,2264.214355,27597.234375,5237.069824,328891.03125,49227.753906,13644.734375,32205.289062,9402.830078,8087.620605,30762.240234,142837.890625,47972.863281,87937.632812,45545.136719,260293.75000,281449.34375,84284.304688,106547.726562,32948.085938,30400.058594,33465.976562,157926.515625,212914.250000,97961.085938,...,39787.621094,107036.976562,291970.37500,13059.770508,165721.765625,182666.750000,7180.042969,7078.189941,34881.199219,13569.545898,77055.726562,51536.554688,51959.246094,26980.919922,295785.18750,612492.5625,721265.4375,21965.425781,58071.007812,67143.820312,24899.738281,120841.960938,37898.144531,13299.776367,3309.182861,39266.402344,18408.347656,67656.710938,100499.945312,5609.979492,51170.046875,51679.628906,87599.523438,37512.738281,27380.882812,56039.503906,312444.6875,262200.062500,26574.640625,17289.794922
2,2021-11-03,29150.873047,35100.992188,60102.648438,149000.093750,102500.250000,17167.527344,48511.078125,84490.046875,99885.570312,674597.3125,50446.667969,251499.734375,34923.031250,55367.656250,17850.318359,2190.813721,26365.287109,5129.566895,324253.06250,49250.035156,13500.192383,30977.695312,9493.653320,7835.135254,29826.710938,142205.703125,47927.496094,88852.937500,44966.906250,255491.12500,280556.12500,84924.609375,104791.007812,32517.878906,29899.796875,33345.671875,158866.312500,213582.750000,98992.617188,...,40212.316406,106905.687500,285100.59375,12955.209961,166703.843750,174005.390625,7270.070801,7010.657715,34260.359375,13544.387695,76642.046875,50724.976562,52651.648438,26495.748047,305476.68750,612865.1250,710464.0000,21744.500000,58509.562500,69704.765625,27199.718750,121674.132812,36918.765625,13399.675781,3265.387695,39932.605469,18120.650391,65453.832031,99100.000000,5560.624023,49951.859375,50528.437500,87899.554688,38185.375000,27104.802734,53022.277344,329814.4375,260700.125000,25808.613281,17447.972656
3,2021-11-04,29339.898438,35462.390625,60238.929688,148000.046875,103000.031250,17109.638672,49010.910156,84537.312500,104054.117188,675035.1875,52046.695312,236000.218750,34424.429688,57287.449219,17199.994141,2282.636475,25702.507812,5180.525879,320674.25000,49470.078125,13349.890625,32647.013672,965